In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime
#import sqlalchemy as sa

In [2]:
sys.path.append(r"../../src/collectcube")
from db import *
from db_post import *

In [3]:
#out_dir = '../../data/LUCinLA_paraguay/smSamp'
#local_db_path = os.path.join(out_dir, 'landcover.db')
out_dir = 'C:/GISprojects/ParaguayValidation/DistrictSample'
local_db_path = 'C:/GISprojects/ParaguayValidation/smDistricts/landcover.db'

In [140]:
table_check = check_table(local_db_path,'pixels')

          PID  Center     cent_X     cent_Y  cent_lat  cent_long  ransamp  \
72606  8606_0       1  3117727.0 -3241221.0       NaN        NaN        0   
72607  8607_0       1  3117233.0 -3240552.0       NaN        NaN        0   
72608  8608_0       1  3117374.0 -3241372.0       NaN        NaN        0   
72609  8609_0       1  3110514.0 -3250962.0       NaN        NaN        0   
72610  8610_0       1  3114744.0 -3243548.0       NaN        NaN        0   

       checked  PID0  PID1            sampgroup  
72606      NaN  8606     0  smallholderHO_SegGE  
72607      NaN  8607     0  smallholderHO_SegGE  
72608      NaN  8608     0  smallholderHO_SegGE  
72609      NaN  8609     0  smallholderHO_SegGE  
72610      NaN  8610     0  smallholderHO_SegGE  


## Data cleaning (e.g. if pandas import below doesn't work)
Note: need for this should be eliminated as more formatting restrictions are placed in db entry functions

### manipulating table (should not need to do this here)

In [ ]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
## adding a new column to existing table
c.execute("ALTER TABLE PixelVerification ADD COLUMN entry_lev INTEGER")
## filling column in existing table
#c.execute("UPDATE PixelVerification SET entry_lev = 4 WHERE entry_lev IS NULL")
con.commit()      
c.close()

### querying table (e.g. for bad records)

In [8]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("SELECT * FROM PixelVerification  WHERE imgDate = '2021_03-07'")
con.commit()
print(c.fetchone()) 
'''
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("SELECT * FROM pixels WHERE PID0 = '8023'")
con.commit()
print(c.fetchone()) 
c.close()
'''

con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE pixels SET cent_X = '3129504.0' WHERE PID0 ='8023'")
con.commit()
c.close()

('8101_0', 1, 3129534.0, -3245991.0, None, None, 0, None, 8101, 0, 'smallholderHO_GE')


### fixing bad records

In [122]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE PixelVerification SET imgDate = '2022-01-01' WHERE imgDate ='1/1/2022'")
con.commit()
c.close()
'''

'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE pixels SET sampgroup = 'D0112_Concepcion_A' WHERE PID0 ='998'")
con.commit()
c.close()
'''

'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE pixels SET PID = '0008023_0' WHERE PID = '8023_0'")
con.commit()
c.close()
'''


con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("DELETE FROM PixelVerification WHERE PID = '507_0'")
con.commit()
c.close()


'\ncon = sqlite3.connect(local_db_path)\nc = con.cursor()\nc.execute("DELETE FROM PixelVerification WHERE recID >= \'12290\'")\ncon.commit()\nc.close()\n'

## Open db table as pandas dataframe

In [141]:
## This uses sqlalchemy but is prefered because it will make sure that formatting is correct
df = open_data_records_sqlachemy(local_db_path)
df.tail()

,recID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,PercentTree,...,TREEPLANT0,HIGHVEG,TREEPLANT,FOREST,Age,Stability,State,Notes,entry_lev,PIDi
12507,14380,5986,4,2016-03-06,20.0,11.0,5.0,,,,...,100,,,,,--,--,,1,0005986_4
12508,14381,5986,5,2016-03-06,20.0,31.0,6.0,,,,...,,,,,,--,--,,1,0005986_5
12509,14382,5986,0,2024-08-13,70.0,60.0,5.0,,,,...,,,,80,,--,--,,1,0005986_0
12510,14386,5986,4,2024-08-13,70.0,60.0,5.0,,,,...,,,,100,,--,--,,1,0005986_4
12511,14387,5986,5,2024-08-13,20.0,31.0,6.0,,,,...,,,,,,--,--,,1,0005986_5


In [134]:
## This uses only sqllite and can be used to examine table if the above fails
conn = sqlite3.connect(local_db_path, isolation_level=None,
    detect_types=sqlite3.PARSE_COLNAMES)
db_df = pd.read_sql_query("SELECT * FROM PixelVerification", conn)
db_df.tail()
#db_df.to_csv(os.path.join('C:/GISprojects/ParaguayValidation/smDistricts','sm_test_db.csv'), index=False)

,recID,PID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,...,MEDVEG,TREEPLANT0,HIGHVEG,TREEPLANT,FOREST,Age,Stability,State,Notes,entry_lev
12507,14380,5986_4,5986,4,2016-03-06,20.0,11.0,5.0,,,...,,100,,,,,--,--,,1
12508,14381,5986_5,5986,5,2016-03-06,20.0,31.0,6.0,,,...,,,,,,,--,--,,1
12509,14382,5986_0,5986,0,2024-08-13,70.0,60.0,5.0,,,...,,,,,80,,--,--,,1
12510,14386,5986_4,5986,4,2024-08-13,70.0,60.0,5.0,,,...,,,,,100,,--,--,,1
12511,14387,5986_5,5986,5,2024-08-13,20.0,31.0,6.0,,,...,,,,,,,--,--,,1


In [8]:
#addfile = 'C:/GISprojects/ParaguayValidation/lgDistricts/lg_addF.csv'
#pd.read_csv(addfile).to_sql('PixelVerification', conn, if_exists='append', index=False)

1476

## Get sample for target date (direct and indirect)

In [142]:
## Will consider anything 1-June of prior year to 1-June of target_date year as direct observation 
## and interpolate from other observations according to the rules set up and the 'stability' column in LUT
## if sample_type is 'area_est' or 'validation', will only include center pixels

target_date = 2022

sample_type = 'area_est' ## 'area_est',  validation', 'training' 
sample_cats = 'bi_crop'   ## 'bi_tree', 'bi_forest'
lut = '../../data/Class_LUT.csv'
   
all_obs=get_sample_for_date(target_date,local_db_path,sample_type,sample_cats,lut)
    
all_obs.sort_values('PID0')
all_obs.tail(n=10)

there are 6106 observations for selected date
there are 5896 center pixel observations


,recID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,PercentTree,...,TREEPLANT,FOREST,Age,Stability,State,Notes,entry_lev,PIDi,obs_type,LC_UNQ
1342,14253.0,8092,0,2023-03-20,20.0,30.0,8.0,,,,...,,,,--,--,,4,0008092_0,indirect_GE,35.0
12390,14260.0,8093,0,2022-04-08,70.0,80.0,3.0,4,,,...,,80,,--,--,,4,0008093_0,direct_GE,65.0
12395,14265.0,8094,0,2022-01-01,20.0,30.0,8.0,,,,...,,,,--,--,,4,0008094_0,direct_GE,35.0
12397,14267.0,8095,0,2022-03-05,20.0,30.0,8.0,,,,...,,,,--,--,,4,0008095_0,direct_GE,31.0
1345,14276.0,8097,0,2022-03-05,20.0,30.0,8.0,,,,...,,,,--,--,,4,0008097_0,indirect_GE,35.0
1346,14281.0,8098,0,2021-07-22,20.0,30.0,8.0,,,,...,,,,--,--,,4,0008098_0,indirect_GE,35.0
12418,14288.0,8099,0,2021-07-22,40.0,30.0,8.0,,,,...,,,,c - true change,--,,4,0008099_0,direct_GE,40.0
12265,14135.0,8100,0,2022-03-27,20.0,98.0,8.0,,,,...,,,,--,--,,4,0008100_0,direct_GE,23.0
1356,14325.0,8101,0,2022-11-14,99.0,98.0,8.0,,,,...,,,,--,--,,4,0008101_0,indirect_GE,23.0
1316,14143.0,8103,0,2023-06-06,20.0,30.0,5.0,,,,...,,,,--,--,,4,0008103_0,indirect_GE,35.0


## Create point file for confusion matrices

In [136]:
pix = open_pix_info_sqlachemy(local_db_path)

pix_data = pd.merge(all_obs,pix,left_on='PIDi',right_on='PID',how='left')
## format to meet specs used in LUCinSA_helpers
pix_data.rename(columns={"PID0": "OID_", "cent_X": "XCoord", "cent_Y":"YCoord"})
pix_data['LC2'] = pix_data.apply(lambda x: 30 if x['LC'] == 30 else 0, axis=1)
pix_data.to_csv(os.path.join(out_dir,'samp2022_pix.csv'))

## Summarize sample size by sample group

In [143]:
pix = open_pix_info_sqlachemy(local_db_path)
#pix_data = pix_data[pix_data['PID1']==0]
pix_data = pd.merge(all_obs,pix,left_on='PIDi',right_on='PID',how='left')
samp_sum=pix_data.groupby(['sampgroup','LC']).size().reset_index(name='counts')
samp_tot=pix_data.groupby(['sampgroup']).size().reset_index(name='total')
samp = pd.merge(samp_sum,samp_tot,on='sampgroup',how='left')
samp['per'] = samp['counts']/samp['total']
samp[samp['LC']==30.0]

,sampgroup,LC,counts,total,per
0,D0102_Concepcion_B,30.0,73,493,0.148073
3,D0103_Concepcion_H,30.0,25,225,0.111111
6,D0112_Concepcion_A,30.0,15,169,0.088757
9,D0217_SanPedro_C,30.0,124,474,0.261603
12,D0301_Cordillera_C,30.0,21,399,0.052632
15,D0502_Caaguazu_C,30.0,99,402,0.246269
18,D0506_Caaguazu_DJMF,30.0,119,295,0.403390
21,D0518_Caaguazu_3F,30.0,161,486,0.331276
24,D0603_Caazapa_B,30.0,50,387,0.129199
27,D0607_Caazapa_SJN,30.0,41,328,0.125000
